In [18]:
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import multiprocessing
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import pymorphy3
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
from sklearn.preprocessing import OrdinalEncoder

from xgboost import XGBClassifier

multiprocessing.cpu_count()

16

In [19]:

df = pd.read_json("kinopoisk.jsonl", lines=True, convert_dates = ['date'])

df.head()

,part,movie_name,review_id,author,date,title,grade3,grade10,content
0,top250,Блеф (1976),17144,Come Back,2011-09-24,Плакали наши денежки ©,Good,10.0,"\n""Блеф» — одна из моих самых любимых комедий...."
1,top250,Блеф (1976),17139,Stasiki,2008-03-04,None,Good,0.0,\nАдриано Челентано продолжает радовать нас св...
2,top250,Блеф (1976),17137,Flashman,2007-03-04,None,Good,10.0,"\nНесомненно, это один из великих фильмов 80-х..."
3,top250,Блеф (1976),17135,Sergio Tishin,2009-08-17,""" Черное, красное, ерунда это все. Выигрывает ...",Good,0.0,\nЭта фраза на мой взгляд отражает сюжет несом...
4,top250,Блеф (1976),17151,Фюльгья,2009-08-20,"«Он хотел убежать? Да! Блеф, блеф…»",Neutral,7.0,"\n- как пела Земфира, скорее всего, по соверше..."


In [20]:

import re

df['content'] = df['content'].apply(lambda st: re.sub('\n', ' ', st))
df['content'] = df['content'].apply(lambda st: re.sub('\xa0', ' ', st))
df['content'] = df['content'].apply(lambda st: re.sub('\nA', ' ', st))

#re.sub('\xa0', '', st)




## Обработка текста

In [21]:
# Создаём копию датасета для BoW

df_bow = df[['content', 'grade3', 'grade10']].copy()

df_bow.head()


# Убираем знаки препинания
df_bow['content'] = df_bow['content'].apply(lambda text: re.sub(r'[^\w\s]', '', text))
df_bow

,content,grade3,grade10
0,Блеф одна из моих самых любимых комедий Это...,Good,10.0
1,Адриано Челентано продолжает радовать нас сво...,Good,0.0
2,Несомненно это один из великих фильмов 80х го...,Good,10.0
3,Эта фраза на мой взгляд отражает сюжет несомн...,Good,0.0
4,как пела Земфира скорее всего по совершенно ...,Neutral,7.0
...,...,...,...
36586,Ну с чего бы начать Давненько я не писа...,Bad,2.0
36587,Можно начать с того что уже постер к фи...,Bad,1.0
36588,Фильм производства России поэтому многи...,Good,7.0
36589,16 сентября на большие экраны вышел мис...,Bad,0.0


In [22]:
# Как поступить с заголовками?
# 1) Отбросить
# 2) Добавить к основному тексту отзыва
# 3) Сравнить перфомансы
tokenizer = nltk.tokenize.WordPunctTokenizer()

def lower_join_tokenizer(st):
  st=str(st)
  return ' '.join(tokenizer.tokenize(st.lower()))

df_bow['content'] = df_bow['content'].apply(lower_join_tokenizer)


#df_bow.to_csv('bow_kinopoisk.csv')

In [23]:
df_bow

,content,grade3,grade10
0,блеф одна из моих самых любимых комедий этот ф...,Good,10.0
1,адриано челентано продолжает радовать нас свои...,Good,0.0
2,несомненно это один из великих фильмов 80х год...,Good,10.0
3,эта фраза на мой взгляд отражает сюжет несомне...,Good,0.0
4,как пела земфира скорее всего по совершенно др...,Neutral,7.0
...,...,...,...
36586,ну с чего бы начать давненько я не писала отри...,Bad,2.0
36587,можно начать с того что уже постер к фильму цв...,Bad,1.0
36588,фильм производства россии поэтому многие ожида...,Good,7.0
36589,16 сентября на большие экраны вышел мистически...,Bad,0.0


In [29]:
def lemmatize(st):
    morph_analyzer = pymorphy3.MorphAnalyzer()
    return [morph_analyzer.parse(word)[0].normal_form for word in st.split()]

In [28]:
!pip install pymorphy3

In [30]:
%%time
result = Parallel(n_jobs=8, verbose=10)(delayed(lemmatize)(text) for text in df_bow['content'])
df_bow['clean_content'] = result

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Batch computation too fast (0.19772261127393442s.) Setting batch_size=2.
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 108 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 138 tasks      | elapsed:    2.3s
[Parallel(n_jobs=8)]: Done 168 tasks      | elapsed:    2.8s
[Parallel(n_jobs=8)]: Done 202 tasks      | elapsed:    3.2s
[Parallel(n_jobs=8)]: Done 236 tasks      | elapsed:    3.7s
[Parallel(n_jobs=8)]: Done 274 tasks 

CPU times: total: 3.81 s
Wall time: 11min 5s


[Parallel(n_jobs=8)]: Done 36591 out of 36591 | elapsed: 11.1min finished


In [26]:
df_bow.head()

,content,grade3,grade10
0,блеф одна из моих самых любимых комедий этот ф...,Good,10.0
1,адриано челентано продолжает радовать нас свои...,Good,0.0
2,несомненно это один из великих фильмов 80х год...,Good,10.0
3,эта фраза на мой взгляд отражает сюжет несомне...,Good,0.0
4,как пела земфира скорее всего по совершенно др...,Neutral,7.0


In [31]:

nltk.download('stopwords')
stop_list = stopwords.words('russian')



df_bow['clean_content'] = df_bow['clean_content'].apply(lambda x: [item for item in x if item not in stop_list])
df_bow.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,content,grade3,grade10,clean_content
0,блеф одна из моих самых любимых комедий этот ф...,Good,10.0,"[блеф, самый, любимый, комедия, фильм, наверно..."
1,адриано челентано продолжает радовать нас свои...,Good,0.0,"[адриано, челентано, продолжать, радовать, сво..."
2,несомненно это один из великих фильмов 80х год...,Good,10.0,"[несомненно, это, великий, фильм, 80х, год, ис..."
3,эта фраза на мой взгляд отражает сюжет несомне...,Good,0.0,"[фраза, взгляд, отражать, сюжет, несомненно, п..."
4,как пела земфира скорее всего по совершенно др...,Neutral,7.0,"[петь, земфира, скорее, весь, совершенно, пово..."


In [32]:
df_bow.to_pickle('lemmed_df.pkl')

## TF-IDF векторизация

In [13]:
df_bow = pd.read_csv('lemmed_df.csv').drop(columns = ['Unnamed: 0'])
df_bow.head()


,content,grade3,grade10,clean_content
0,блеф одна из моих самых любимых комедий этот ф...,Good,10.0,"['блеф', 'самый', 'любимый', 'комедия', 'фильм..."
1,адриано челентано продолжает радовать нас свои...,Good,0.0,"['адриано', 'челентано', 'продолжать', 'радова..."
2,несомненно это один из великих фильмов 80х год...,Good,10.0,"['несомненно', 'это', 'великий', 'фильм', '80х..."
3,эта фраза на мой взгляд отражает сюжет несомне...,Good,0.0,"['фраза', 'взгляд', 'отражать', 'сюжет', 'несо..."
4,как пела земфира скорее всего по совершенно др...,Neutral,7.0,"['петь', 'земфира', 'скорее', 'весь', 'соверше..."


In [35]:
df_bow = pd.read_pickle('lemmed_df.pkl')

df_bow['clean_content'][0]

['блеф',
 'самый',
 'любимый',
 'комедия',
 'фильм',
 'наверно',
 'смотреть',
 'сто',
 'блефовать',
 'видеть',
 'большой',
 'мочь',
 'выразить',
 'свой',
 'восхищение',
 'главный',
 'действовать',
 'лицо',
 'это',
 'фильм',
 'начать',
 'адриано',
 'челентано',
 'который',
 'считать',
 'это',
 'хороший',
 'роль',
 'кино',
 'великолепный',
 'актёр',
 'неплохой',
 'певец',
 'странно',
 'родина',
 'италия',
 'песня',
 'мало',
 'слушать',
 'думать',
 'итальянец',
 'француз',
 'привыкнуть',
 'сей',
 'актёр',
 'популярный',
 'свой',
 'родина',
 'парадокс',
 'челентано',
 'профессионал',
 'свой',
 'дело',
 'комик',
 'серьёзный',
 'выражение',
 'лицо',
 'смешной',
 'ещё',
 'одновременно',
 'серъёзный',
 'адриано',
 'браво',
 'несколько',
 'слово',
 'энтони',
 'куиня',
 'самый',
 'горбун',
 'нотрдама',
 'собор',
 'парижский',
 'богоматерь',
 'оригинальный',
 'версия',
 'смотреть',
 'рекомендовать',
 'както',
 'приключиться',
 'интересный',
 'история',
 'съёмка',
 'свой',
 'фильм',
 'сломать',
 '

In [36]:
df_bow['clean_content'] = df_bow['clean_content'].str.join(' ')

In [37]:
df_bow['grade3'].value_counts()

grade3
Good       27264
Bad         4751
Neutral     4576
Name: count, dtype: int64

In [38]:
count_vec = CountVectorizer()

count_vec.fit(df_bow['clean_content'])

CountVectorizer()

In [39]:
matrix = count_vec.fit_transform(df_bow['clean_content'].values)
freqs = zip(count_vec.get_feature_names_out(), matrix.sum(axis=0))    
# sort from largest to smallest
A = np.squeeze(np.asarray(matrix.sum(axis=0)))
freqs = zip(count_vec.get_feature_names_out(), A)    
sorted(freqs, key=lambda x: -x[1])[:10]

[('фильм', 177682),
 ('это', 142066),
 ('который', 83746),
 ('всё', 77457),
 ('свой', 64003),
 ('весь', 55049),
 ('человек', 50215),
 ('очень', 41916),
 ('10', 41407),
 ('мочь', 38706)]

In [40]:
df_bow_balanced = df_bow[df_bow['grade3']=='Good'].sample(n = 4600, random_state=42)

df_bow_balanced = pd.concat([df_bow_balanced, df_bow[df_bow['grade3']!='Good']]).reset_index(drop = True)



In [41]:


reviews = df_bow_balanced['clean_content'].values



tfidf = TfidfVectorizer(max_features=1500)

tfidf.fit(df_bow['clean_content'].values)

X = tfidf.transform(reviews)
X.shape

(13927, 1500)

In [42]:

OE = OrdinalEncoder(categories=[['Bad', 'Neutral', 'Good']])
y = OE.fit_transform(df_bow_balanced['grade3'].values.reshape(-1, 1))

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [44]:
params = {
        'min_child_weight': [1, 5, 10],
        'max_depth': [4, 5, 6]
        }

In [45]:
%%time
XGBC = XGBClassifier(n_jobs = 4)

XGBC.fit(X_train, y_train)

predictions = XGBC.predict(X_test)

#Calculating accuracy


CPU times: total: 1min 27s
Wall time: 34.2 s


In [46]:
from sklearn.model_selection import GridSearchCV  
from dask.distributed import Client, LocalCluster 
import joblib

In [48]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

grid_search = GridSearchCV(XGBC, param_grid=params, cv = 5, scoring = 'accuracy', verbose=10)

# create a local Dask cluster
'''
cluster = LocalCluster()  

client = Client(cluster)  



with joblib.parallel_backend("dask", scatter=[X_train, y_train]):  

    grid_search.fit(X_train, y_train)

'''
# Here we go
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=4, min_child_weight=1.............................
[CV 1/5; 1/9] END max_depth=4, min_child_weight=1;, score=0.663 total time=  15.6s
[CV 2/5; 1/9] START max_depth=4, min_child_weight=1.............................
[CV 2/5; 1/9] END max_depth=4, min_child_weight=1;, score=0.652 total time=  15.1s
[CV 3/5; 1/9] START max_depth=4, min_child_weight=1.............................
[CV 3/5; 1/9] END max_depth=4, min_child_weight=1;, score=0.669 total time=  15.9s
[CV 4/5; 1/9] START max_depth=4, min_child_weight=1.............................
[CV 4/5; 1/9] END max_depth=4, min_child_weight=1;, score=0.685 total time=  14.6s
[CV 5/5; 1/9] START max_depth=4, min_child_weight=1.............................
[CV 5/5; 1/9] END max_depth=4, min_child_weight=1;, score=0.663 total time=  14.6s
[CV 1/5; 2/9] START max_depth=4, min_child_weight=5.............................
[CV 1/5; 2/9] END max_depth=4, min_chil

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=4, num_parallel_tree=None,
                                     objective='multi:softprob', ...),
             param_grid={'max_depth': [4, 5, 6],
                         'min_child_weight': [1, 5, 10]},
             scoring='accuracy', verbose=10)

In [254]:
random_search.best_score_

nan

In [237]:
accuracy = accuracy_score(y_test, predictions)
accuracy

0.6734635267087881

In [238]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.70      0.73      0.72      1183
         1.0       0.55      0.57      0.56      1143
         2.0       0.77      0.72      0.74      1156

    accuracy                           0.67      3482
   macro avg       0.68      0.67      0.67      3482
weighted avg       0.68      0.67      0.67      3482



In [203]:
feature_array = np.array(tfidf.get_feature_names_out())
tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]

In [204]:
top_n

array(['сон', 'подруга', 'крутой'], dtype=object)

CountVectorizer()

[('00', matrix([[  7, 346,   1, ...,   1,   1,   1]], dtype=int64))]

[('фильм', 177682),
 ('это', 142066),
 ('который', 83746),
 ('всё', 77457),
 ('свой', 64003),
 ('весь', 55049),
 ('человек', 50215),
 ('очень', 41916),
 ('10', 41407),
 ('мочь', 38706),
 ('хороший', 38411),
 ('просто', 33923),
 ('жизнь', 33892),
 ('герой', 30559),
 ('ещё', 28284),
 ('актёр', 27346),
 ('время', 25200),
 ('самый', 25121),
 ('смотреть', 24680),
 ('каждый', 22976),
 ('роль', 22011),
 ('картина', 21613),
 ('первый', 21422),
 ('сказать', 21421),
 ('история', 20838),
 ('кино', 20409),
 ('сюжет', 20334),
 ('большой', 19162),
 ('игра', 18833),
 ('мир', 17927),
 ('год', 17694),
 ('просмотр', 17225),
 ('посмотреть', 16809),
 ('главный', 16360),
 ('стать', 16219),
 ('говорить', 15513),
 ('зритель', 15225),
 ('друг', 15186),
 ('любовь', 14896),
 ('знать', 14809),
 ('режиссёр', 14779),
 ('персонаж', 14628),
 ('видеть', 13953),
 ('конец', 13857),
 ('именно', 13853),
 ('наш', 12925),
 ('хотеть', 12856),
 ('момент', 12470),
 ('хотя', 12423),
 ('сделать', 12314),
 ('слово', 12066),
 ('н